In [34]:
import torch
import torchvision
import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import cv2 as cv
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as T
from PIL import Image, ImageDraw


In [35]:
# https://towardsdatascience.com/everything-about-fasterrcnn-6d758f5a6d79
# https://www.kaggle.com/code/dipam7/wheat-head-classification/notebook

In [36]:
boxes = pd.read_csv("rescaled_data.csv")
boxes.head()

,Unnamed: 0,number,jsonname,photoname,width,height,class,xmin,ymin,xmax,ymax,xmin_scaled,xmax_scaled,ymin_scaled,ymax_scaled
0,1150,194032,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,2688,3911,m,580.707317,731.707317,785.585366,892.682927,108.018474,146.128230,65.481350,79.887247
1,1151,194032,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,2688,3911,m,561.195122,1219.512195,663.634146,1329.268293,104.388974,123.443852,109.135584,118.957786
2,990,184970,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3366,4866,m,2525.969697,2348.484848,2944.151515,2627.272727,375.218315,437.336826,168.921023,188.973583
3,991,184970,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3366,4866,m,2604.757576,2969.696970,3016.878788,3287.878788,386.921803,448.140046,213.603358,236.489432
4,992,184970,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3366,4866,m,1695.666667,2500.000000,2235.060606,2627.272727,251.881561,332.005438,179.819153,188.973583


In [37]:
boxes = pd.read_csv("rescaled_data.csv")
boxes = boxes[["number", "xmin_scaled", "ymin_scaled", "xmax_scaled", "ymax_scaled"]]
boxes.head()

,number,xmin_scaled,ymin_scaled,xmax_scaled,ymax_scaled
0,194032,108.018474,65.481350,146.128230,79.887247
1,194032,104.388974,109.135584,123.443852,118.957786
2,184970,375.218315,168.921023,437.336826,188.973583
3,184970,386.921803,213.603358,448.140046,236.489432
4,184970,251.881561,179.819153,332.005438,188.973583


In [38]:
boxes.tail()

,number,xmin_scaled,ymin_scaled,xmax_scaled,ymax_scaled
1703,184415,346.255886,125.962763,466.169443,150.890595
1704,184549,298.020978,205.516706,397.297561,291.958042
1705,186537,112.028066,177.358083,135.097020,187.276911
1706,186537,84.171215,268.280669,184.717035,292.841575
1707,186537,436.734481,226.007571,461.979752,245.845226


In [39]:
def preprocessing(imageID):
    """reads in image and returns preprocessed np array"""
    img = cv.imread(f"./data/rescaled_png_files/{imageID}.png")
   # img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = img/255
    img = torch.tensor(img, dtype=torch.float32)
    img = img.permute(2,0,1) # change channel position
    return img


In [40]:
image_list = os.listdir('./data/rescaled_png_files/')

In [42]:
name = "19394.png"
name.removesuffix(".png")

'19394'

In [41]:
data = []
all_box_coordinates = []
for image in image_list:
    image_dict = {}
    id = image.removesuffix('.png')  
    sub_df = boxes[boxes["number"] == int(id)]
    num_boxes = len(sub_df)
    box_coordinates = []
    for i in range(num_boxes):
        sub_sub_df = sub_df.iloc[i]
        xmin_scaled = int(sub_sub_df["xmin_scaled"])
        ymin_scaled = int(sub_sub_df["ymin_scaled"])
        xmax_scaled = int(sub_sub_df["xmax_scaled"])
        ymax_scaled = int(sub_sub_df["ymax_scaled"])
        box_coordinates.append(torch.tensor([xmin_scaled, ymin_scaled, xmax_scaled, ymax_scaled], dtype=torch.int32))
    if num_boxes > 1:
        box_coordinates = torch.stack(box_coordinates, axis=0)
    else:
        box_coordinates = box_coordinates[0]
        box_coordinates = box_coordinates.view(1,4)

    all_box_coordinates.append(box_coordinates)
    
    image_data = preprocessing(id) # returns list

    # labels
    labels = torch.ones(num_boxes, dtype=torch.int64)

    # stack it to dict
    image_dict["data"] = image_data
    image_dict["boxes"] = box_coordinates
    image_dict["labels"] = labels
    image_dict["image_id"] = id

    data.append(image_dict)

In [16]:
class MarginaliaDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        self.n_samples = len(data)

    def __getitem__(self, index):
        img = self.data[index]["data"]
        boxes = self.data[index]["boxes"]
        labels = self.data[index]["labels"]
        id = self.data[index]["image_id"]
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        return img, target, id
    
    def __len__(self):
        return self.n_samples

In [17]:
train_data = data[:4]
test_data = data[len(data)-4:]

In [18]:
train_dataset = MarginaliaDataset(train_data)
test_dataset = MarginaliaDataset(test_data)

In [19]:
img, target, id = train_dataset[13]
print(img.shape)
print(id)

IndexError: list index out of range

In [ ]:
print(target["boxes"])

tensor([[212,  17, 259,  35],
        [291,  19, 355,  29],
        [ 64,  33, 458, 121],
        [ 77, 129, 122, 148],
        [382, 221, 441, 241],
        [ 43, 222, 129, 251]], dtype=torch.int32)


In [20]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dl = DataLoader(train_dataset, batch_size=4, num_workers=4, collate_fn=collate_fn, pin_memory = True)
val_dl = DataLoader(test_dataset, batch_size=4, collate_fn=collate_fn, pin_memory = True)

In [21]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = 2  # 1 class (marginalia) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model=model.to(device)

In [39]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
num_epochs = 1

In [40]:
model.to(device)
for epoch in range(num_epochs):
    epoch_loss = 0
    for images, targets, _ in train_dl:
        optimizer.zero_grad()
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
#         print(loss_dict)
        losses = sum(loss for loss in loss_dict.values())
        epoch_loss += losses.item()

        losses.backward()
        optimizer.step()
    print(f"loss for epoch {epoch}: {epoch_loss / len(train_dl)}")

: 

: 

In [1]:
#torch.save(model, 'faster_r_cnn.pth')

In [ ]:
results=[]
detection_threshold = 0.5 # ???
model.eval()
model.to(device)

for images, targets, id in val_dl:    

    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes']
        scores = outputs[i]['scores']
        labels = outputs[i]['labels']

        keep = torchvision.ops.nms(boxes, scores, 0.1) # the lower, the less we keep
        boxes = boxes[keep]
        scores = scores[keep]
        image_id = id[i]
    
        op = (id[i], boxes, scores)
        results.append(op)

        #break
    #break

print(results)

In [ ]:
def visualize_prediction(imageID, tensor_bounding_box):
    tensor_bounding_box = tensor_bounding_box.cpu().numpy()
    image = cv.imread(f"data/{imageID}.png")
    image = np.asarray(image)
    for box in tensor_bounding_box:
        x_min = box[0]
        y_min = box[1]
        x_max = box[2]
        y_max = box[3]
        print(x_min)
        
        color = (0, 0, 255)
        start_point = (int(x_min), int(y_min))
        end_point = (int(x_max), int(y_max))
        thickness = 2
        cv.rectangle(image, start_point, end_point, color, thickness)
    cv.imwrite(f'results/prediction_{imageID}.png', image)

In [33]:
for images, targets, id in val_dl:
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    outputs = model(images)
    for i, image in enumerate(images):
        visualize_prediction(id, outputs)

ValueError: too many values to unpack (expected 2)